# Fast implementation of levenshtein distance
* Approximation rapide de levenshtein distance
* Se base sur l'algorithme de norvig

In [1]:
import re
from collections import Counter
import cPickle
import operator

In [2]:
#charge le corpus
corpus = file('data/big.txt').read()
all_words = file('data/words.txt.1').read()
alphabet = 'abcdefghijklmnopqrstuvwxyz'

# Extraction des mots

In [3]:
def find_all_words(text):
    text = text.replace("\n","")
    text.replace("\t"," ")
    mots = [a.lower() for a in text.split(" ") if len(a)>0]
    return mots
def find_all_words2(text):
    mots = [a.lower() for a in text.split("\n")]
    return mots

mots = find_all_words(corpus) + find_all_words2(all_words)
counted_words = Counter(mots)
print("Nombre de mots differents : ",sum(counted_words.values()))

('Nombre de mots differents : ', 1356950)


# Algorithme
* Calcule la correction d'un mot
* Gere les erreurs d'insertion, de deletion et de changement
* Calcule la distance entre deux mots
* Calcule a une distance de deux maximum
(cf rapport)

In [4]:
def in_dict(list_mots):
    #verifie si un mot a deja ete vu
    return {w for w in list_mots if w in counted_words}

#gere les suppresions, par exemple : test -> tet
def deletions(word):
    return [word[:i] + word[i+1:] for i in range(len(word))]

#gere les remplacements, par exemple : 
# test -> tast
def remplacements(word):
    return [word[:i] + k + word[i+1:] for k in alphabet for i in range(len(word))]

#gere les insertions par exemple : test -> teast
def insertions(word):
    return [word[:i] + c +word[i:] for c in alphabet for i in range(len(word)+1)]

#gere la transpositions de caracteres
def transpoitions(word):
    return [a+b[1]+b[0]+b[2:] for (a, b) in [(word[:i], word[i:]) for i in range(len(word)+1)] if len(b) > 1]

#genere tous les mots possibles a une distance de 1
def distance_one(word):
    return set(remplacements(word)+ transpoitions(word) + insertions(word) + deletions(word))

#cherche les candidats en generant toutes les erreurs possibles
#et en cherchant les mots ayant le plus de similarites
def find_candidates(mot,edit_distance = 2):
    candidates = []
    #distance de 0 : mot lui meme
    candidates = in_dict({mot})
    #mots a une distance d'edition de 1
    if len(candidates)==0:
        candidates = {w for w in distance_one(mot) if w in counted_words}
    if len(candidates)==0:
        motif = [new_w for m in distance_one(mot) for new_w in distance_one(m)]
        candidates = {w for w in motif if w in counted_words}
    candidates = set(candidates)
    if len(candidates)==0:
        return mot
    return max(candidates, key=counted_words.get)

# Test sur nos donnees
* Fonctionne sur des phrases entières

# Calcul des ngrams

In [5]:
#calcul des bigrams:
def ngrams(mot, n):
    output = []
    for i in range(len(mot)-n+1):
        output.append(mot[i:i+n])
    return output

bigrams = {x.lower():ngrams(x.lower(),2) for x in counted_words.keys()}
trigrams = {x.lower():ngrams(x.lower(),3) for x in counted_words.keys()}
quadrugrams = {x.lower():ngrams(x.lower(),4) for x in counted_words.keys()}

def jaccard_distance(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def brute_force_find(mot):
    bigram_mot = ngrams(mot,3)
    res = {}
    for e in trigrams:
        res[e] = jaccard_distance(bigram_mot,trigrams[e])
    sorted_res = sorted(res.items(), key=operator.itemgetter(1),reverse=True)
    print(sorted_res[:10])
brute_force_find("test")  

[('test', 1.0), ('test,', 0.6666666666666666), ('test.', 0.6666666666666666), ('tests', 0.6666666666666666), ('testy', 0.6666666666666666), ('testa', 0.6666666666666666), ('teste', 0.6666666666666666), ('testes', 0.6666666666666666), ('gestes', 0.5), ('testis', 0.5)]


# Calcule des similarités

In [7]:
#les tests
with open("typos-data/test10.pkl", "rb") as input_file:
    test = cPickle.load(input_file)
#les donnes d'entrainement    
with open("typos-data/train20.pkl", "rb") as input_file:
    train = cPickle.load(input_file)
errors= 0
total = 0
nb_errors_hard = 0
#des erreurs que le hmm d'ordre 2 n'arrive pas a traiter
hard = ["reeljhgs","vakhavle","cillectivism","maaochistic","lertusts","pfoyesr","masochixtic","ofersocialkzed","pwfrixt","sctiviwts","empyiomap","frudyrarwd","leftustx"]
#sur l'ensemble de test
for t in test:
    to_decod = "".join([a[0] for a in t])
    etiquette = [a[1] for a in t]
    if len(to_decod)>0:
        solution = find_candidates(to_decod)
        errors += sum([a!=b for a,b in zip(solution,etiquette)])
        if sum([a!=b for a,b in zip(solution,etiquette)])>0:
            nb_errors_hard+=1
    total += len(etiquette)
print(" Taux d'erreurs : {}% \n Nombre d'erreurs :{} \n Nombre de tests : {} \n Nombre d'erreurs ayant une distance d'édition >2 : {} ".format((errors/float(total))*100.0, errors,len(test),nb_errors_hard))
    


 Taux d'erreurs : 4.3306010929% 
 Nombre d'erreurs :317 
 Nombre de tests : 1501 
 Nombre d'erreurs ayant une distance d'édition >2 : 174 
